# Tutorial: EAGERx Advanced usage

In the remainder of this tutorial, we will go more into detail on these concepts.

Furthermore, you will be asked to add/modify a couple of lines of code, which are marked by

```python

# YOUR CODE HERE

# END OF YOUR CODE
```

## Activate GPU (Colab only)

When in Colab, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package and ROS.render_fn="disc_pendulum_render_fn",render_fn="disc_pendulum_render_fn",

In [1]:
try:
    import eagerx_tutorials
except ImportError:
    !{"echo 'Installing eagerx-tutorials with pip.' && pip install eagerx-tutorials >> /tmp/eagerx_install.txt 2>&1"}
if 'google.colab' in str(get_ipython()):
    !{"curl 'https://raw.githubusercontent.com/eager-dev/eagerx_tutorials/master/scripts/setup_colab.sh' > ~/setup_colab.sh"}
    !{"bash ~/setup_colab.sh"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()

Not running on CoLab.
Execute ROS commands as "!...".
ROS noetic available.


## Let's get started

First we will import EAGERx and initialize it.
EAGERx makes use of ROS functionality for communication and during initialization a ROS master is started if there isn't one running already.

In [2]:
import eagerx
# Initialize eagerx (starts roscore if not already started.)
eagerx.initialize("eagerx_core", log_level=eagerx.log.INFO)

... logging to /home/r2ci/.ros/log/696bfb52-d78f-11ec-a85f-83d2f95e85fe/roslaunch-r2ci-Alienware-m15-R4-5752.log
started roslaunch server http://192.168.68.129:41803/
ros_comm version 1.15.14


SUMMARY

PARAMETERS
 * /rosdistro: noetic
 * /rosversion: 1.15.14

NODES

auto-starting new master
process[master]: started with pid [5849]
ROS_MASTER_URI=http://localhost:11311
setting /run_id to 696bfb52-d78f-11ec-a85f-83d2f95e85fe
process[rosout-1]: started with pid [5874]
started core service [/rosout]


In [3]:
import numpy as np

episode_timeout = 10  # [s]
desired_velocity = 20  # [deg/s]
env_rate = 50  # [Hz]
cpg_rate = 200  # [Hz]
cartesian_rate = 200  # [Hz]
quad_rate = 200  # [Hz]
sim_rate = 200  # [Hz]
# sensors = ["pos", "vel", "base_orientation", "base_pos", "base_vel", "force_torque"]  # Todo: works
# sensors = ["pos", "vel", "base_orientation", "base_pos", "base_vel"]
sensors = ["base_orientation", "force_torque"]

In [4]:
# Initialize empty graph
graph = eagerx.Graph.create()

# Create robot
import eagerx_tutorials.quadruped.object  # noqa: F401
robot = eagerx.Object.make(
    "Quadruped",
    "quadruped",
    actuators=["joint_control"],
    sensors=sensors,
    rate=quad_rate,
    control_mode="position_control",
    self_collision=False,
    fixed_base=False,
)
robot.sensors.pos.rate = env_rate
robot.sensors.vel.rate = env_rate
robot.sensors.force_torque.rate = env_rate
robot.sensors.base_orientation.rate = env_rate
robot.sensors.base_pos.rate = env_rate
robot.sensors.base_vel.rate = env_rate
graph.add(robot)

# Create cartesian control node
import eagerx_tutorials.quadruped.cartesian_control  # noqa: F401
cartesian_control = eagerx.Node.make("CartesiandPDController", "cartesian_control", rate=cartesian_rate, process=eagerx.process.ENVIRONMENT)
graph.add(cartesian_control)

# Create cpg node
import eagerx_tutorials.quadruped.cpg_gait  # noqa: F401
gait = "TROT"
omega_swing, omega_stance = {
    "JUMP": [4 * np.pi, 40 * np.pi],
    "TROT": [16 * np.pi, 4 * np.pi],
    "WALK": [16 * np.pi, 4 * np.pi],
    "PACE": [20 * np.pi, 20 * np.pi],
    "BOUND": [10 * np.pi, 20 * np.pi],
}[gait]
cpg = eagerx.Node.make(
    "CpgGait",
    "cpg",
    rate=cpg_rate,
    gait=gait,
    omega_swing=omega_swing,
    omega_stance=omega_stance,
    process=eagerx.process.ENVIRONMENT,
)
graph.add(cpg)

pybullet build time: Apr 26 2022 03:12:14


In [5]:
# Connect the nodes
graph.connect(action="offset", target=cpg.inputs.offset)
graph.connect(source=cpg.outputs.cartesian_pos, target=cartesian_control.inputs.cartesian_pos)
graph.connect(source=cartesian_control.outputs.joint_pos, target=robot.actuators.joint_control)
if "position" in sensors:
    graph.connect(observation="position", source=robot.sensors.pos)
if "velocity" in sensors:
    graph.connect(observation="velocity", source=robot.sensors.vel)
if "position" in sensors:
    graph.connect(observation="base_pos", source=robot.sensors.base_pos)
if "force_torque" in sensors:
    graph.connect(observation="force_torque", source=robot.sensors.force_torque)
if "base_vel" in sensors:
    graph.connect(observation="base_vel", source=robot.sensors.base_vel)
assert "base_orientation" in sensors, "The base_orientation must always be included in the sensors, " \
                                      "because it is used to calculate the reward."
graph.connect(observation="base_orientation", source=robot.sensors.base_orientation, window=2)  # window=2
if "xs_zs" in sensors:
    graph.connect(
        observation="xs_zs",
        source=cpg.outputs.xs_zs,
        skip=True,
        initial_obs=[-0.01354526, -0.26941818, 0.0552178, -0.25434446],
    )

In [6]:
# Define engine
engine = eagerx.Engine.make(
    "PybulletEngine",
    rate=sim_rate,
    gui=False,
    egl=False,
    sync=True,
    real_time_factor=0,
    process=eagerx.process.ENVIRONMENT,
)

In [7]:
from typing import Dict, Tuple
from eagerx.wrappers import Flatten
import pybullet
import gym

print("TEST")
class QuadrupedEnv(eagerx.BaseEnv):
    def __init__(self, name, rate, graph, engine, desired_velocity, episode_timeout, force_start=True, debug=False):
        super(QuadrupedEnv, self).__init__(name, rate, graph, engine, force_start=force_start)
        self.steps = None
        self.debug = debug
        self.timeout_steps = int(episode_timeout * rate)

    @property
    def observation_space(self) -> gym.spaces.Dict:
        return self._observation_space

    @property
    def action_space(self) -> gym.spaces.Dict:
        return self._action_space

    def reset(self):
        # Reset number of steps
        self.steps = 0

        # Sample desired states
        states = self.state_space.sample()

        # Perform reset
        obs = self._reset(states)
        return obs

    def step(self, action: Dict) -> Tuple[Dict, float, bool, Dict]:
        obs = self._step(action)
        self.steps += 1

        # Calculate reward
        alive_bonus = 0.25

        # Convert Quaternion to Euler
        _, _, prev_yaw = pybullet.getEulerFromQuaternion(obs["base_orientation"][-2])
        roll, pitch, yaw = pybullet.getEulerFromQuaternion(obs["base_orientation"][-1])

        # Current angular velocity
        yaw_rate = (yaw - prev_yaw) * env_rate
        desired_yaw_rate = np.deg2rad(desired_velocity)

        # yaw_cost = np.linalg.norm(yaw_rate - desired_yaw_rate)
        yaw_cost = (yaw_rate - desired_yaw_rate) ** 2
        reward = alive_bonus - yaw_cost

        if self.debug:
            # print(len(obs["base_vel"][0]), len(obs["velocity"][0]))
            print(yaw_cost)
            # print(obs["base_vel"][0][:2])

        # print(list(map(np.rad2deg, (roll, pitch, yaw))))
        has_fallen = abs(np.rad2deg(roll)) > 40 or abs(np.rad2deg(pitch)) > 40
        timeout = self.steps >= self.timeout_steps

        # Determine done flag
        done = timeout or has_fallen
        # Set info about episode truncation
        info = {"TimeLimit.truncated": timeout and not has_fallen}
        return obs, reward, done, info
print("TEST2")

TEST
TEST2


In [8]:
# Initialize Environment
# Unique ID to be able to launch multiple instances
print("TEST")
env = QuadrupedEnv(
    name=f"QuadrupedEnv",
    rate=env_rate,
    graph=graph,
    engine=engine,
    desired_velocity=desired_velocity,
    episode_timeout=episode_timeout,
    debug=False,
)
print("END CREATING ENV")
env = Flatten(env)

TEST
[INFO] [1652977160.022773]: Node "/QuadrupedEnv/env/supervisor" initialized.


EGL device choice: -1 of 4.


[INFO] [1652977160.342478]: Node "/QuadrupedEnv/engine" initialized.
[INFO] [1652977160.476087]: Node "/QuadrupedEnv/environment" initialized.
[INFO] [1652977160.547189]: Node "/QuadrupedEnv/cartesian_control" initialized.
TROT
[INFO] [1652977160.583163]: Node "/QuadrupedEnv/cpg" initialized.
END CREATING ENV
[INFO] [1652977160.658448]: Adding object "quadruped" of type "Quadruped" to the simulator.
/home/r2ci/eagerx-dev/eagerx_tutorials/eagerx_tutorials/quadruped/go1/go1_description/urdf/go1.urdf
[INFO] [1652977161.434254]: Waiting for nodes "['quadruped/force_torque_sensor']" to be initialized.
[INFO] [1652977161.452010]: Node "/QuadrupedEnv/quadruped/force_torque_sensor" initialized.
[INFO] [1652977161.464705]: Node "/QuadrupedEnv/quadruped/base_orientation_sensor" initialized.
[INFO] [1652977161.476809]: Node "/QuadrupedEnv/quadruped/joint_control" initialized.


In [10]:
from sb3_contrib import TQC
# Initialize SB3
hyperparams = dict(
    learning_rate=1e-3,
    tau=0.02,
    gamma=0.98,
    buffer_size=300000,
    learning_starts=100,
    use_sde=True,
    use_sde_at_warmup=True,
    train_freq=8,
    gradient_steps=10,
    verbose=1,
    top_quantiles_to_drop_per_net=0,
    policy_kwargs=dict(n_critics=1, net_arch=dict(pi=[64, 64], qf=[64, 64])),
)

model = TQC("MlpPolicy", env, **hyperparams)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [13]:
print("START TRAINING")
model.learn(1_000_000)

START TRAINING
[step] KEYBOARD INTERRUPT


KeyboardInterrupt: 